In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-dataset.zip

--2022-08-04 23:42:32--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-dataset.zip [following]
--2022-08-04 23:42:32--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc42e908383cdc0fdebce7d9dbb3.dl.dropboxusercontent.com/cd/0/inline/BqZQiRq4tLiXQRx55hWtCquAn74MN27c6QG1umsl5N_GpkJhdJ97qLbsWbtYPMpjFvq70odcVwNQ8Kpj5x7TQSK3qlgyqvpOUYAufhKPrB5jOcssBHf29cd-J5u8eUdPP7AX3zcRvf3SC_BsjQOQUG6GHV_zdCz2Tk2uij5rBW4Cvg/file# [following]
--2022-08-04 23:42:32--  https://uc42e908383cdc0fdebce7d9dbb3.dl.dropboxusercontent.com/cd/0/inline/BqZQiRq4tLiXQRx55hWtCquAn74MN27c6QG1umsl5N_GpkJhdJ97q

In [2]:
!unzip -q 'movie-dataset.zip'

In [5]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, SimpleRNNCell, Embedding, Dropout, GRU

In [4]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, '<br />',' '), label),)

In [6]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'SNL is pretty funny but people who say this is like watching a Short skit on SNL is a little dumb minded. It\'s NOTHING like SNL, it\'s just a stupid piece of crap.  Andy Samberg tries to act like Jon Heder but fails. Although Jon Heder is only funny in Napoleon Dynamite Andy tries his hardest and people think he\'s funny.  Only funny people in the movie were Danny McBride and Bill Hader. The only part that was decently funny with Andy was the pool part.  They could have made the "Quiet place" a lot better if they didn\'t make the falling scene 3 freaking minutes.  The part where he\'s pronouncing his H\'s more is retarded. They try going with it too long and half the time it looks like Andy is laughing while he acts...he\'s a horrible actor and doesn\'t deserve to be in a movie.  This movie is a joke and is for the simple minded people with the brain of a 10 year old level of comedy. Which is about h

In [7]:
model = Sequential()
model.add(Input(shape=(1,),dtype='string'))

In [8]:
max_tokens = 1000
max_len = 100

vectorized_layer = TextVectorization(max_tokens = max_tokens,
                                     output_mode = 'int',
                                     output_sequence_length = max_len,)





In [9]:
train_text = train_data.map(lambda text, label : text)
vectorized_layer.adapt(train_text)

model.add(vectorized_layer)

In [10]:
# creating an embedding layer

model.add(Embedding(max_tokens + 1, 128))
# rnn = RNN(SimpleRNNCell(64), return_sequences = False, return_state = False) # can change it from here
model.add(GRU(64))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [11]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [12]:
model.fit(train_data, epochs = 10)

Epoch 1/10
782/782 [==============================] - 56s 68ms/step - loss: 0.5542 - accuracy: 0.6942
Epoch 2/10
782/782 [==============================] - 53s 68ms/step - loss: 0.4126 - accuracy: 0.8122
Epoch 3/10
782/782 [==============================] - 53s 68ms/step - loss: 0.3830 - accuracy: 0.8257
Epoch 4/10
782/782 [==============================] - 55s 70ms/step - loss: 0.3648 - accuracy: 0.8354
Epoch 5/10
782/782 [==============================] - 55s 70ms/step - loss: 0.3481 - accuracy: 0.8426
Epoch 6/10
782/782 [==============================] - 55s 70ms/step - loss: 0.3311 - accuracy: 0.8500
Epoch 7/10
782/782 [==============================] - 55s 70ms/step - loss: 0.3125 - accuracy: 0.8585
Epoch 8/10
782/782 [==============================] - 55s 70ms/step - loss: 0.2953 - accuracy: 0.8650
Epoch 9/10
782/782 [==============================] - 55s 71ms/step - loss: 0.2738 - accuracy: 0.8738
Epoch 10/10
782/782 [==============================] - 55s 70ms/step - loss: 0.255

In [13]:
model.evaluate(test_data)

782/782 [==============================] - 17s 22ms/step - loss: 0.5901 - accuracy: 0.7817


[0.5901255011558533, 0.7816799879074097]

In [14]:
model.predict(['I loved the movie'])

array([[0.96383846]], dtype=float32)